In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import os
from google.colab import files

projectID = 'finalproject-ecmms'
location = 'US'
client = bigquery.Client(project=projectID, location=location)

auth.authenticate_user()

print('authenticated')

In [ ]:
queries = {
    '2-best_clients_2023': """
        SELECT u.id,
          u.first_name,
          u.last_name,
          u.email,
          ROUND(SUM(oi.sale_price)) AS totale_spesa,
          COUNT(DISTINCT o.order_id) AS count_orders_per_client
        FROM `bigquery-public-data.thelook_ecommerce.orders` AS o
        JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
          ON o.user_id = u.id
        JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi
          ON oi.order_id = o.order_id
        WHERE oi.status = 'Complete'AND oi.created_at >= TIMESTAMP('2023-01-01')
        GROUP BY u.id, u.first_name, u.last_name, u.email
        ORDER BY totale_spesa DESC
    """,
    '2-users_countries': """
        SELECT
          CASE
            WHEN country IN ('Germany', 'Deutschland') THEN 'Germany'
            ELSE country
          END AS country,
          city,
          COUNT(*) AS frequency
        FROM `bigquery-public-data.thelook_ecommerce.users`
        GROUP BY country, city
        ORDER BY frequency DESC
    """,
    '2-users_gender_age': """
        SELECT
          gender,
          COUNT(*) AS total,
          COUNT(CASE WHEN age < 18 THEN 1 END) AS under_18,
          COUNT(CASE WHEN age BETWEEN 18 AND 30 THEN 1 END) AS between_18_30,
          COUNT(CASE WHEN age BETWEEN 30 AND 50 THEN 1 END) AS between_30_50,
          COUNT(CASE WHEN age > 50 THEN 1 END) AS over_50
        FROM `bigquery-public-data.thelook_ecommerce.users`
        GROUP BY gender
    """
}

for query_name, query in queries.items():
    query_job = client.query(query)
    results = query_job.result().to_dataframe()
    filename = f'{query_name}.csv'
    results.to_csv(filename, index=False)
    print(f'Risultati "{query_name}" salvati come CSV')
    files.download(filename)

print('Download completato')
